<a href="https://colab.research.google.com/github/anuthereaper/PythonLibrary/blob/main/Storage_queue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install azure.storage.queue

In [ ]:
pip install aiohttp

In [ ]:
#Write a single message to to storage queue using storage key 
from azure.storage.queue import (
        QueueClient,
        BinaryBase64EncodePolicy,
        BinaryBase64DecodePolicy
)
#import os, uuid
import asyncio
from azure.storage.queue.aio import QueueClient as QueueClientasync 


accountname = "anustorage1980"
accountkey = <account_key>
queue_name = "myqueue" 
msg_per_page = 10

connect_str = "DefaultEndpointsProtocol=https;AccountName=" + accountname + ";AccountKey=" + accountkey + ";EndpointSuffix=core.windows.net"
q_name = queue_name
queue_client = QueueClient.from_connection_string(connect_str, q_name)

# Setup Base64 encoding and decoding functions
base64_queue_client = QueueClient.from_connection_string(
                            conn_str=connect_str, queue_name=q_name,
                            message_encode_policy = BinaryBase64EncodePolicy(),
                            message_decode_policy = BinaryBase64DecodePolicy())
def Queue_send(message):
    print("Adding message: " + message)
    queue_client.send_message(message)

def Queue_send_b64(message):
    print("Adding b64 message: " + message)  
    base64_queue_client.send_message(bytes(message,'utf-8'))

def Queue_receive():
    response = queue_client.receive_messages()
    for message in response:
        print("Received message : " + message.content)
        queue_client.delete_message(message)

def Queue_receive_inbatch(msg_per_page):
    response = queue_client.receive_messages(messages_per_page=msg_per_page)

    for message_batch in response.by_page():
        for message in message_batch:
            print("Received paged message : " + message.content)
            queue_client.delete_message(message)


Queue_send("{'OU' : 'UK', 'DB' : 'Shell_uk', 'TEST' : 'NORMAL_SEND'}")
Queue_send_b64("{'OU' : 'UK', 'DB' : 'Shell_uk', 'TEST' : 'B64_SEND'}")
Queue_receive()
Queue_send("{'OU' : 'UK', 'DB' : 'Shell_uk', 'TEST' : 'PAGED_SEND'}")
Queue_receive_inbatch(msg_per_page)

In [ ]:
pip install names

In [ ]:
# SEND MANY MESSAGES TO QUEUE
from random import randrange
import random
import names
import json
from datetime import datetime
from datetime import timedelta
from azure.storage.queue import (
        QueueClient,
        BinaryBase64EncodePolicy,
        BinaryBase64DecodePolicy
)
import uuid

accountname = "anustorage1980"
accountkey = <account_key>
q_name = "myqueue" 
Number_of_docs = 10
d1 = datetime.strptime('1/1/2008 1:30 PM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('1/1/2009 4:50 AM', '%m/%d/%Y %I:%M %p')

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def Create_Record(item_id):
    """
    This function will return a service bus message with random data.
    """
    doc_body = {'id' : str(uuid.uuid4()),
            'employee_id' : str(id),
            'first_name' : names.get_first_name(),
            'last_name' : names.get_last_name(),
            'regn_no' : 'TEST00000000' + str(item_id),
            'start_date' : str(random_date(d1,d2)),
            'salary' : str(random.randint(12000, 2000000)),
            'inserted_at' : str(datetime.now())
            }
    json_object = json.dumps(doc_body, indent = 4) 
    return(json_object)
#    return bytes(json_object,'utf-8')

connect_str = "DefaultEndpointsProtocol=https;AccountName=" + accountname + ";AccountKey=" + accountkey + ";EndpointSuffix=core.windows.net"
q_name = queue_name
queue_client = QueueClient.from_connection_string(connect_str, q_name)
# Setup Base64 encoding and decoding functions
base64_queue_client = QueueClient.from_connection_string(
                            conn_str=connect_str, queue_name=q_name,
                            message_encode_policy = BinaryBase64EncodePolicy(),
                            message_decode_policy = BinaryBase64DecodePolicy())

starttime = datetime.utcnow()
print("Starting ingestion: ", starttime.strftime("%Y-%m-%d %H:%M:%S.%f"))
for i in range (Number_of_docs):
    message = Create_Record(i)
#    queue_client.send_message(message)
    base64_queue_client.send_message(bytes(message,'utf-8'))
    if (i+1)%100 == 0 or (i == Number_of_docs-1):
         print(str(i+1) + " documents completed")

endtime = datetime.utcnow()
print("\nrun_sample done :" + endtime.strftime("%Y-%m-%d %H:%M:%S.%f"))
print("Time taken :" + str(endtime-starttime))
print("Number of messages :" + str(Number_of_docs))

# New Section